## Setup

All libs that will be use in this note

In [1]:
import numpy as np
from glob import glob
import random
import time 

## Preprocessing

There are two folders that contains the inputs of the problem **Traveling Salesman Problem** they are different so let's split it.

### Test Data

In [2]:
PATH_TEST = 'instancias/instancias_teste/'
DATA_TEST = glob(PATH_TEST+'*.txt')
print(DATA_TEST)

['instancias/instancias_teste/ch150.txt', 'instancias/instancias_teste/bays29.txt', 'instancias/instancias_teste/berlin52.txt', 'instancias/instancias_teste/ch130.txt', 'instancias/instancias_teste/brazil58.txt', 'instancias/instancias_teste/swiss42.txt', 'instancias/instancias_teste/bayg29.txt', 'instancias/instancias_teste/bier127.txt', 'instancias/instancias_teste/descricao.txt']


### TSP Cup Data

In [3]:
PATH_CUP = 'instancias/instancias_tsp_cup/'
DATA_CUP = glob(PATH_CUP+'*.txt')
print(DATA_CUP)

['instancias/instancias_tsp_cup/tsp1.txt', 'instancias/instancias_tsp_cup/tsp2.txt', 'instancias/instancias_tsp_cup/tsp3.txt', 'instancias/instancias_tsp_cup/descricao.txt']


## Math

Some mathematics functions that will be useful 

In [4]:
def euclidian_distance(p1, p2):
    return int(np.linalg.norm(p1-p2).round())

In [5]:
print(euclidian_distance(np.array([1,1]), np.array([1,10])))

9


## Graph

We'll use a graph as data structure in order to representation our problem, where each city is a vertex and a connection is representation by a edge. 

In [6]:
class Graph:
    def __init__(self, num_vertices):
        self._adjacency_matrix = np.zeros((num_vertices, num_vertices), dtype=np.int64)
        
    def build(self, file, _type='tsp_test'):
        if _type == 'tsp_cup':
            self._handle_tsp_cup(file)
        elif _type == 'tsp_test':
            self._handle_tsp_test(file)
        else:
            print('Undefined type')
        
    def _handle_tsp_cup(self, file):
        data = []
        with open(file, 'r') as f:
            for line in f:
                try:
                    _, x, y = list(filter(lambda x: x!='', line.split(' ')))
                    data.append(np.array((float(x), float(y[:-1]))))
                except:
                    pass
        for i, p1 in enumerate(data):
            for j, p2 in enumerate(data):
                self._adjacency_matrix[i][j] = euclidian_distance(p1, p2)
    
    def _handle_tsp_test(self, file):
        data = []
        with open(file, 'r') as f:
            _input = f.readlines()
            _input = [_.strip() for _ in _input]
            _input = [_.split() for _ in _input]
            for i, array in enumerate(_input[3:]):
                try:
                    _ = [int(_) for _ in array]
                    if len(_) > 0:
                        data.append(_)
                except:
                    pass
        self._adjacency_matrix = data
    
    def num_vertices(self):
        return len(self._adjacency_matrix)
    
    def get_cost(self, _list_path_nodes):
        path = [self._adjacency_matrix[current][_next] for current, _next in zip(_list_path_nodes[1:], _list_path_nodes[:-1])]
        return sum(path)

## Optimizations Solutions

Some functions that will give us solutions for our problem.

## Helpers

Below functions that will be userful to generation solutions

In [7]:
def pick_element(_list, nodes):
    for index, x in enumerate(_list):
        if not nodes[index]:
            return index, x
    return None

In [8]:
def get_min_distance(_distances, nodes):
    min_index, min_val = pick_element(_distances, nodes)
    for index, x in enumerate(_distances):
        if min_val > x:
            min_val = x if not nodes[index] else min_val
            min_index = index if not nodes[index] else min_index
    return min_val, min_index

In [9]:
def get_cost_swap(graph, cost, _list, i, k):
    cost_swap = cost
    if i == k:
        return cost_swap
    if i > k:
        i, k = k, i
    if i+1 == k:
        prev, _next = i-1 if i > 0 else i, k+1 if k < graph.num_vertices() - 1 else k
        cost_swap -= graph.get_cost([_list[prev], _list[i], _list[k], _list[_next]])
        
        prev, _next = i-1 if i > 0 else k, k+1 if k < graph.num_vertices() - 1 else i
        cost_swap += graph.get_cost([_list[prev], _list[k], _list[i], _list[_next]])
    else:
        prev_i, _next_i = i-1 if i > 0 else i, i+1
        prev_k, _next_k = k-1, k+1 if k < graph.num_vertices() - 1 else k
        
        cost_swap -= graph.get_cost([_list[prev_i], _list[i], _list[_next_i]])
        cost_swap -= graph.get_cost([_list[prev_k], _list[k], _list[_next_k]])
        
        prev_i, _next_i = i-1 if i > 0 else k, i+1
        prev_k, _next_k = k-1, k+1 if k < graph.num_vertices() - 1 else i
        
        cost_swap += graph.get_cost([_list[prev_i], _list[k], _list[_next_i]])
        cost_swap += graph.get_cost([_list[prev_k], _list[i], _list[_next_k]])
    return cost_swap

### Random Restart

In [10]:
def random_restart(graph, itermax):
    _iter = 0
    solution = [i for i in range(graph.num_vertices())]
    np.random.shuffle(solution)
    current_solution, current_cost = solution, graph.get_cost(solution)
    while _iter < itermax:
        np.random.shuffle(solution)
        cost = graph.get_cost(solution)
        if current_cost > cost:
            current_cost, current_solution = cost, solution
            _iter = 0
        else:
            _iter += 1
    current_solution = current_solution + [current_solution[0]]
    return graph.get_cost(current_solution), current_solution

### Nearest Neighbor

In [11]:
def nearest_neighbor(graph, initial):
    solution = []
    current_node = initial
    nodes = [False for x in range(graph.num_vertices())]
    nodes[current_node] = True
    flag = True
    solution.append(current_node)
    while flag:
        distances = graph._adjacency_matrix[current_node]
        min_distance, min_index = get_min_distance(distances, nodes)
        current_node = min_index
        nodes[current_node] = True
        solution.append(current_node)
        if all(nodes):
            solution.append(initial)
            flag = False
    return graph.get_cost(solution), solution

### Best Insertion

In [12]:
def best_insertion(graph, initial):
    current_cost, current_solution = initial
    index = np.random.randint(graph.num_vertices())
    for i in range(graph.num_vertices()):
        cost = get_cost_swap(graph, current_cost, current_solution, i, index)
        if current_cost > cost:
            current_solution[i], current_solution[index] = current_solution[index], current_solution[i]
            current_cost = cost
    return graph.get_cost(current_solution), current_solution

### Swap

In [13]:
def first_swap(graph, initial):
    current_cost, current_solution = initial
    flag = True
    while flag:
        flag = False
        for i in range(graph.num_vertices()):
            for k in range(i+1, graph.num_vertices()):
                cost = get_cost_swap(graph, current_cost, current_solution, i, k) 
                if current_cost > cost:
                    current_solution[i], current_solution[k] = current_solution[k], current_solution[i]
                    current_cost = cost
                    return graph.get_cost(current_solution), current_solution
    return graph.get_cost(current_solution), current_solution

### Perturbation

In [14]:
def perturbation(graph, _list):
    i=k=0
    while i==k:
        i, k = np.random.randint(graph.num_vertices()), np.random.randint(graph.num_vertices())
    if i > k:
        i, k = k, i
    start, middle, end = _list[:i], _list[i:k+1], _list[k+1:]
    middle.reverse()
    solution = start + middle + end
    return graph.get_cost(solution), solution

### VND - Variable Neighborhood Descent

In [15]:
def vnd(graph, initial, itermax):
    _iter = 0
    current_cost, current_solution = graph.get_cost(initial), initial
    while _iter < itermax:
        is_better = True
        while is_better:
            is_better = False
            cost, solution = best_insertion(graph, (current_cost, current_solution))
            if current_cost > cost:
                current_cost, current_solution = cost, solution
                is_better = True
                _iter = 0
            else:
                cost, solution = first_swap(graph, (current_cost, current_solution))
                if current_cost > cost:
                    current_cost, current_solution = cost, solution
                    is_better = True
                    _iter = 0
        _iter += 1
    return graph.get_cost(current_solution), current_solution

### GRASP - Greedy Randomized Adaptive Search Procedure 

In [16]:
def grasp(graph, itermax, alpha=0):
    _iter = 0
    current_cost = current_solution = -1
    while _iter < itermax:
        _random = np.random.random_sample() + alpha
        _, solution = nearest_neighbor(graph, np.random.randint(graph.num_vertices())) if _random < 1 else random_restart(graph, 100)
        solution = solution[:-1]
        cost, solution = vnd(graph, solution, 1)
        if current_cost > cost or _iter == 0:
            current_cost, current_solution = cost, solution
        _iter += 1
    current_solution = current_solution + [current_solution[0]]
    return graph.get_cost(current_solution), current_solution

### VNS - Variable Neighborhood Search

In [17]:
def vns(graph, itermax):
    _iter = 0
    current_cost = current_solution = -1
    _, solution = nearest_neighbor(graph, np.random.randint(graph.num_vertices()))
    solution = solution[:-1]
    while _iter < itermax:
        _, solution = perturbation(graph, solution)
        cost, solution = vnd(graph, solution, 1)
        if current_cost > cost or _iter == 0:
            current_cost, current_solution = cost, solution
        _iter += 1
    current_solution = current_solution + [current_solution[0]]
    return graph.get_cost(current_solution), current_solution

## Test

For each test we'll build a graph with the adjacency matrix representing our cities.

## Helpers

Below functions that will be userful to generation informations about our solutions

In [18]:
def gap(val_heur, optimal):
    return ((val_heur-optimal)/optimal)*100

In [19]:
def nearest_neighbor_test(graph, iter_max, optimal):
    indexes = []
    costs = []
    times = []
    random_index = np.random.randint(graph.num_vertices())
    indexes.append(random_index)
    _iter = 0
    while _iter < iter_max:
        while random_index in indexes:
            random_index = np.random.randint(graph.num_vertices())
        indexes.append(random_index)
        start = time.time()
        cost, solution = nearest_neighbor(graph, random_index)
        end = time.time()
        times.append(end-start)
        costs.append(cost)
        print('({:}) cost: {:} time: {:}'.format(_iter+1, cost, end-start))
        _iter += 1
    print('Mean Cost: {:} \t Optimal: {:} \t Mean Time: {:} \t Gap: {:}'
          .format(np.mean(costs), np.min(costs), np.mean(times), gap(np.min(costs),optimal)))

In [20]:
def grasp_test(graph, iter_max, optimal):
    _iter = 0
    costs = []
    times = []
    while _iter < iter_max:
        start = time.time()
        cost, solution = grasp(graph, 50, alpha=0.5)
        end = time.time()
        times.append(end-start)
        costs.append(cost)
        print('({:}) cost: {:} time: {:}'.format(_iter+1, cost, end-start))
        _iter += 1
    print('Mean Cost: {:} \t Optimal: {:} \t Mean Time: {:} \t Gap: {:}'
          .format(np.mean(costs), np.min(costs), np.mean(times), gap(np.min(costs),optimal)))

In [21]:
def vns_test(graph, iter_max, optimal):
    _iter = 0
    costs = []
    times = []
    while _iter < iter_max:
        start = time.time()
        cost, solution = vns(graph, 50)
        end = time.time()
        times.append(end-start)
        costs.append(cost)
        print('({:}) cost: {:} time: {:}'.format(_iter+1, cost, end-start))
        _iter += 1
    print('Mean Cost: {:} \t Optimal: {:} \t Mean Time: {:} \t Gap: {:}'
          .format(np.mean(costs), np.min(costs), np.mean(times), gap(np.min(costs),optimal)))

### TSP TEST

In [22]:
ITER_MAX = 10
print(DATA_TEST)

['instancias/instancias_teste/ch150.txt', 'instancias/instancias_teste/bays29.txt', 'instancias/instancias_teste/berlin52.txt', 'instancias/instancias_teste/ch130.txt', 'instancias/instancias_teste/brazil58.txt', 'instancias/instancias_teste/swiss42.txt', 'instancias/instancias_teste/bayg29.txt', 'instancias/instancias_teste/bier127.txt', 'instancias/instancias_teste/descricao.txt']


### ch150

In [ ]:
graph = Graph(150)
graph.build(DATA_TEST[0], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
nearest_neighbor_test(graph, ITER_MAX, 6528)

#### Greedy Randomized Adaptive Search Procedure

In [ ]:
grasp_test(graph, ITER_MAX, 6528)

#### Variable Neighborhood Search

In [ ]:
vns_test(graph, ITER_MAX, 6528)

### bays29

In [ ]:
graph = Graph(29)
graph.build(DATA_TEST[1], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
nearest_neighbor_test(graph, ITER_MAX, 2020)

#### Greedy Randomized Adaptive Search Procedure

In [ ]:
grasp_test(graph, ITER_MAX, 2020)

#### Variable Neighborhood Search

In [ ]:
vns_test(graph, ITER_MAX, 2020)

### berlin52

In [ ]:
graph = Graph(52)
graph.build(DATA_TEST[2], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
nearest_neighbor_test(graph, ITER_MAX, 7542)

#### Greedy Randomized Adaptive Search Procedure

In [ ]:
grasp_test(graph, ITER_MAX, 7542)

#### Variable Neighborhood Search

In [ ]:
vns_test(graph, ITER_MAX, 7542)

### ch130

In [ ]:
graph = Graph(130)
graph.build(DATA_TEST[3], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
nearest_neighbor_test(graph, ITER_MAX, 6110)

#### Greedy Randomized Adaptive Search Procedure

In [ ]:
grasp_test(graph, ITER_MAX, 6110)

#### Variable Neighborhood Search

In [ ]:
vns_test(graph, ITER_MAX, 6110)

### brazil58

In [ ]:
graph = Graph(58)
graph.build(DATA_TEST[4], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
nearest_neighbor_test(graph, ITER_MAX, 25395)

#### Greedy Randomized Adaptive Search Procedure

In [ ]:
grasp_test(graph, ITER_MAX, 25395)

#### Variable Neighborhood Search

In [ ]:
vns_test(graph, ITER_MAX, 25395)

### swiss42

In [ ]:
graph = Graph(42)
graph.build(DATA_TEST[5], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
nearest_neighbor_test(graph, ITER_MAX, 1273)

#### Greedy Randomized Adaptive Search Procedure

In [ ]:
grasp_test(graph, ITER_MAX, 1273)

#### Variable Neighborhood Search

In [ ]:
vns_test(graph, ITER_MAX, 1273)

### bayg29

In [ ]:
graph = Graph(29)
graph.build(DATA_TEST[6], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
nearest_neighbor_test(graph, ITER_MAX, 1610)

#### Greedy Randomized Adaptive Search Procedure

In [ ]:
grasp_test(graph, ITER_MAX, 1610)

#### Variable Neighborhood Search

In [ ]:
vns_test(graph, ITER_MAX, 1610)

### bier127

In [ ]:
graph = Graph(127)
graph.build(DATA_TEST[7], _type='tsp_test')
print(graph.num_vertices())

#### Nearest Neighbor

In [ ]:
nearest_neighbor_test(graph, ITER_MAX, 118282)

#### Greedy Randomized Adaptive Search Procedure

In [ ]:
grasp_test(graph, ITER_MAX, 118282)

#### Variable Neighborhood Search

In [ ]:
vns_test(graph, ITER_MAX, 118282)